# Introdução

In [ ]:
import os
import re
import time
import json
import folium
import random
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from folium import plugins
from osgeo import gdal, osr
from bs4 import BeautifulSoup
from tqdm.notebook import trange, tqdm

<br>

# Join Data

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp_defesacivil/main/data/tabs/tab_municipio_defesacivil.csv',
)

# Lê o arquivo csv com o nome dos municípios
gdf = gpd.read_file(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/shps/sp_250k_wgs84.geojson',
)
gdf.drop(['municipio_nome'], axis=1, inplace=True)
gdf['id_municipio'] = gdf['id_municipio'].astype(int)
gdf['geometry'] = gdf.simplify(0.0015)

# Merge
gdf = gdf.merge(
    df,
    on='id_municipio',
    how='left'
)

# Results
gdf.head()

<br>

## Create pop up's column

In [ ]:
# Add Field
def popup_html(row):
    tel_0 = str(row['coord_regional_tel']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_1 = str(row['coord_adj1_tel']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_2 = str(row['coord_adj2_tel']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_3 = str(row['coord_adj3_tel']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_4 = str(row['coord_adj4_tel']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_5 = str(row['coord_adj5_tel']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_6 = str(row['coord_adj6_tel']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel_7 = str(row['coord_adj7_tel']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    
    
    html = """
    <div>
    <p><b>{}</b> pertence à:
    <h4>REPDEC de <b>{}</b></h4></p>
    <p><b>Coordenador:</b><br>
    {}<br>
    {}<br>
    {}<br>
    <br>
    <a id="seeMore" onclick="toggleSeeMore()" href="javascript:void(0);">Outros Coordenadores</a><p>    
    <p id="textarea" style="display: none">
    Coordenador Adjunto #01<br>
    Coordenador Adjunto #02<br>
    Coordenador Adjunto #03<br>
    Coordenador Adjunto #04<br>
    Coordenador Adjunto #05<br>
    Coordenador Adjunto #06<br>
    Coordenador Adjunto #07<br>
    </p>
    
    </div>
    """.format(
        '' if row['municipio_nome'] is np.NaN else '{}'.format(row['municipio_nome']),
        '' if row['repdec_regiao'] is np.NaN else '{}'.format(row['repdec_regiao']),
        '' if row['coord_regional_nome'] is np.NaN else '{}'.format(row['coord_regional_nome']),
        '' if row['coord_regional_mail'] is np.NaN else 'E-mail: <a href="mailto:{}">{}</a>'.format(row['coord_regional_mail'], row['coord_regional_mail']),
        '' if row['coord_regional_tel'] is np.NaN else 'Telefone: <a href="tel:{}">{}</a>'.format(tel_0, row['coord_regional_tel']),
    )
    
    html = html.replace('\n','')    
    html = re.sub('\s\s+' , ' ', html) # Remove Espaços no meio
    html = html.strip()
    return html

In [ ]:
# Calculate PopUps
gdf['popup'] = gdf.apply(popup_html, axis=1)

<br>

## Adjust Table

In [ ]:
# Delete Columns
gdf.drop([
    'id_municipio', 'repdec_nome', 
    'coord_regional_nome', 'coord_regional_mail', 'coord_regional_tel',
    'coord_adj1_nome', 'coord_adj1_mail', 'coord_adj1_tel',
    'coord_adj2_nome', 'coord_adj2_mail', 'coord_adj2_tel',
    'coord_adj3_nome', 'coord_adj3_mail', 'coord_adj3_tel',
    'coord_adj4_nome', 'coord_adj4_mail', 'coord_adj4_tel',
    'coord_adj5_nome', 'coord_adj5_mail', 'coord_adj5_tel',
    'coord_adj6_nome', 'coord_adj6_mail', 'coord_adj6_tel',
    'coord_adj7_nome', 'coord_adj7_mail', 'coord_adj7_tel',
], axis=1, inplace=True)
print(gdf.columns)

# Save geojson
gdf.to_file(
    os.path.join('data', 'shps', 'sp_defesacivil.geojson'),
    driver='GeoJSON',
    encoding='utf-8'
)

# Results
gdf.head()

## Get Centroid

In [ ]:
def get_centroid(gdf):
    gdf['apenasparacentroid'] = 35
    gdf_dissolve = gdf.dissolve(by='apenasparacentroid')
    gdf_centroid = gdf_dissolve.representative_point()
    gdf = gdf.drop('apenasparacentroid', axis=1)
    return [float(gdf_centroid.y), float(gdf_centroid.x)]

In [ ]:
list_centroid = get_centroid(gdf)
list_centroid

## Folium

In [ ]:
def map_bomb(input_geojson, bbox):
    gdf = gpd.read_file(input_geojson)
    
    # Column with category
    col_categories = 'repdec_regiao'
    
    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))

    # Create Map
    m = folium.Map(
        location=[-22.545968889465207, -49.56185387118866],
        zoom_start=6,
        min_zoom=6,
        max_zoom=11,
        max_bounds=True,
        min_lon = bbox['min_lon']*(101/100),
        max_lon = bbox['max_lon']*(99/100),        
        min_lat = bbox['min_lat']*(101/100),
        max_lat = bbox['max_lat']*(99/100),
        #zoom_delta=0.1,
    )
    folium.GeoJson(
        gdf,
        name='Defesa Civil',
        smooth_factor=1.0,
        zoom_on_click=False,
        embed=False,
        style_function=lambda x: {
            'fillColor': color_polygon[x['properties'][col_categories]],
            'color': color_polygon[x['properties'][col_categories]],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'repdec_regiao'],
            aliases=['Munícipio', 'REPDEC'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup'],
            parse_html=False,
            max_width='300',
            show=False,
            labels=False,
            sticky=True,            
        )        
    ).add_to(m)

    # Plugins
    m.fit_bounds(m.get_bounds())
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    return m

In [ ]:
# Map without Bounds
bbox = {
    'max_lat': 0,
    'max_lon': 0,
    'min_lat': 0,
    'min_lon': 0,
}
m = map_bomb(os.path.join('data', 'shps', 'sp_defesacivil.geojson'), bbox)

# Map with Bounds
bbox = {
    'max_lat': m.get_bounds()[1][0],
    'min_lat': m.get_bounds()[0][0],
    'max_lon': m.get_bounds()[1][1],
    'min_lon': m.get_bounds()[0][1],
}
m = map_bomb(os.path.join('data', 'shps', 'sp_defesacivil.geojson'), bbox)

# Results
print(bbox)
m

In [ ]:
def add_script(map_folium):
    """
    """
    import folium
    import branca as bc
    
    # Header to Add
    head = """
    {% macro header(this, kwargs) %}
    <script>
    function toggleSeeMore() {
    if(document.getElementById("textarea").style.display == 'none') {
    document.getElementById("textarea").style.display = 'block';
    document.getElementById("seeMore").innerHTML = '...';
    }
    else {
    document.getElementById("textarea").style.display = 'none';
    document.getElementById("seeMore").innerHTML = 'Outros coordenadores';
    }
    }
    </script>
    {% endmacro %}
    """
    
    # Add Header
    macro = bc.element.MacroElement()
    macro._template = bc.element.Template(head)
    map_folium.get_root().add_child(macro)
    return map_folium

In [ ]:
m = add_script(m)
m

<br>

# Save Map

In [ ]:
os.makedirs('maps', exist_ok=True)
m.save(os.path.join('maps', 'defesacivil_map.html'))
m.save(os.path.join('..', '..', '..', 'case_django', 'divadmin', 'templates', 'defesacivil_map.html'))